In [ ]:
!pip install ddi-fw==0.0.239

In [ ]:
import torch
import os

# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

torch.cuda.is_available()

In [ ]:
from google.colab import userdata

In [ ]:
token = userdata.get("GITHUB_API_KEY")
username = userdata.get("GITHUB_USER_NAME")
repo = 'enhancing-ddi-classification-via-textual-arguments'
path = '/content/'+ repo

In [ ]:
import os

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [ ]:
!git clone https://{token}@github.com/{username}/{repo}

In [ ]:
%cd {repo}

In [ ]:
!pwd

In [ ]:
!pip freeze

In [ ]:
db_zip_path = f'{path}/data/drugbank/db/zips'
db = f'{db_zip_path}/drugbank.db'

In [ ]:
from ddi_fw.utils import ZipHelper,Py7ZipHelper

zip_helper = ZipHelper()
zip_helper.extract(input_path=str(db_zip_path), output_path=str(db_zip_path))

In [ ]:
# !pip install python-stopwatch
from ddi_fw.datasets.db_utils import create_connection

In [ ]:
import pandas as pd
import numpy as np
def __select_all_drugs_as_dataframe__(conn):

        cur = conn.cursor()
        cur.execute(f'''
            select
            _Drugs."index",
            drugbank_id,
            _Drugs.name,
            description,
            synthesis_reference,
            indication,
            pharmacodynamics,
            mechanism_of_action,
            toxicity,
            metabolism,
            absorption,
            half_life,
            protein_binding,
            route_of_elimination,
            volume_of_distribution,
            clearance,
            smiles,
            smiles_morgan_fingerprint,
            enzymes_polypeptides,
            targets_polypeptides,
            pathways,
            tuis_description

            from _Drugs


    ''')
        #  where
        #         targets_polypeptides is not null and
        #         enzymes_polypeptides is not null and
        #         pathways is not null and
        #         smiles_morgan_fingerprint is not null
         #        drugbank_id in {format(param)} and


        # pathway is absent

        rows = cur.fetchall()
        headers = ['index', 'id', 'name', 'description', 'synthesis_reference', 'indication', 'pharmacodynamics', 'mechanism_of_action', 'toxicity', 'metabolism', 'absorption', 'half_life',
                   'protein_binding', 'route_of_elimination', 'volume_of_distribution', 'clearance', 'smiles_notation', 'smile', 'enzyme', 'target', 'pathway','tui_description']
        df = pd.DataFrame(columns=headers, data=rows)
        df['smile'] = df['smile'].apply(lambda x:
                                        np.fromstring(
                                            x.replace(
                                                '\n', '')
                                            .replace('[', '')
                                            .replace(']', '')
                                            .replace('  ', ' '), sep=','))
        df['enzyme'] = df['enzyme'].apply(
            lambda x: x.split('|') if x is not None else [])
        df['target'] = df['target'].apply(
            lambda x: x.split('|') if x is not None else [])
        df['pathway'] = df['pathway'].apply(
            lambda x: x.split('|') if x is not None else [])
        df['tui_description'] = df['tui_description'].apply(
            lambda x: x.split('|') if x is not None else [])
        return df

In [ ]:
conn = create_connection(db)

In [ ]:
drugs = __select_all_drugs_as_dataframe__(conn)
drugs.head()

In [ ]:
columns = ['description', 'synthesis_reference', 'indication',
           'pharmacodynamics', 'mechanism_of_action',
           'toxicity', 'metabolism',
           'absorption', 'half_life',
           'protein_binding', 'route_of_elimination',
           'volume_of_distribution', 'clearance']

In [ ]:
drugs['all_text'] = drugs[columns].apply(lambda x: ' '.join(x.dropna()), axis=1)

In [ ]:
drugs.columns

In [ ]:
from ddi_fw.langchain import DataFrameToVectorDB,generate_embeddings

In [ ]:
import uuid
def generate_uuid():
  return str(uuid.uuid4())

In [ ]:
# generate_uuid()

In [ ]:
import json
config_file = f'{path}/embedding_configurations/embeddings_config.json'
with open(config_file, 'r') as f:
    collections = json.load(f)

In [ ]:
collections

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
import torch

# Set the fraction of memory to allocate (e.g., 50% of available memory)
# torch.cuda.set_per_process_memory_fraction(0.5, device=0)  # device=0 for the first GPU


In [ ]:
generate_embeddings(drugs, config_file)

In [ ]:
# from ddi_fw.utils import create_kaggle_dataset

In [ ]:
# !chmod 600 ~/.kaggle/kaggle.json
create_kaggle_dataset("embeddings", collections)